Cell 1 — Konfigurasi

In [1]:
from pathlib import Path

# =========================
# EDIT DI SINI SAJA
# =========================
N_TARGET = 56          # ganti: 48 / 56 / 64
SUBJECT  = "J"         # ganti: "A".."J"
FILE     = "Jalan19.csv"

DATASET_ROOT = Path(r"E:\0.TA_Teguh\GMM Trial 2")
HEAD1_DIR    = DATASET_ROOT / "Head 1"

FPS_ROOT     = Path(r"E:\0.TA_Teguh\Sampling\FPS")
WRS_ROOT     = Path(r"E:\0.TA_Teguh\Sampling\Weighted")

COLS = ["timestamp","frame","x","y","z","doppler","SNR","w_snr","d2","is_inlier"]

print("Config:")
print("N_TARGET:", N_TARGET)
print("SUBJECT :", SUBJECT)
print("FILE    :", FILE)
print("HEAD1   :", HEAD1_DIR)
print("FPS_ROOT:", FPS_ROOT)
print("WRS_ROOT:", WRS_ROOT)


Config:
N_TARGET: 56
SUBJECT : J
FILE    : Jalan19.csv
HEAD1   : E:\0.TA_Teguh\GMM Trial 2\Head 1
FPS_ROOT: E:\0.TA_Teguh\Sampling\FPS
WRS_ROOT: E:\0.TA_Teguh\Sampling\Weighted


Cell 2 — Load 3 file

In [2]:
import pandas as pd

def read_csv_strict(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(str(path))
    df = pd.read_csv(path)
    missing = [c for c in COLS if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns in {path}: {missing}")
    return df[COLS].copy()

base_path = HEAD1_DIR / SUBJECT / FILE
fps_path  = FPS_ROOT / f"N{N_TARGET}" / SUBJECT / FILE
wrs_path  = WRS_ROOT / f"N{N_TARGET}" / SUBJECT / FILE

print("Paths:")
print("BASE:", base_path)
print("FPS :", fps_path)
print("WRS :", wrs_path)

df_base = read_csv_strict(base_path)
df_fps  = read_csv_strict(fps_path)
df_wrs  = read_csv_strict(wrs_path)

# frame range (pakai baseline sebagai sumber kebenaran)
fmin = int(df_base["frame"].min())
fmax = int(df_base["frame"].max())
print("Frame range:", fmin, "to", fmax)

# quick sanity: pastikan fps/wrs punya frame column
print("Rows (baseline/fps/wrs):", len(df_base), len(df_fps), len(df_wrs))


Paths:
BASE: E:\0.TA_Teguh\GMM Trial 2\Head 1\J\Jalan19.csv
FPS : E:\0.TA_Teguh\Sampling\FPS\N56\J\Jalan19.csv
WRS : E:\0.TA_Teguh\Sampling\Weighted\N56\J\Jalan19.csv
Frame range: 2 to 250
Rows (baseline/fps/wrs): 9477 12488 12488


Cell 3 — Helper: ambil frame + axis union + ringkas centroid

In [3]:
import numpy as np

def get_frame(df: pd.DataFrame, frame_id: int) -> pd.DataFrame:
    return df[df["frame"] == frame_id].copy()

def compute_union_lims(dfs, margin_ratio=0.10):
    pts = []
    for d in dfs:
        if d is not None and len(d) > 0:
            pts.append(d[["x","y","z"]].to_numpy(dtype=np.float32))
    if not pts:
        return ((-1,1),(-1,1),(-1,1))

    P = np.vstack(pts)
    mn = P.min(axis=0)
    mx = P.max(axis=0)
    span = (mx - mn) + 1e-6
    margin = span * margin_ratio
    mn2 = mn - margin
    mx2 = mx + margin
    return ((float(mn2[0]), float(mx2[0])),
            (float(mn2[1]), float(mx2[1])),
            (float(mn2[2]), float(mx2[2])))

def summarize_frame(df: pd.DataFrame):
    if df is None or len(df) == 0:
        return {"n": 0, "mu_x": None, "mu_y": None, "mu_z": None}
    xyz = df[["x","y","z"]].to_numpy(dtype=np.float32)
    mu = xyz.mean(axis=0)
    return {"n": int(len(df)), "mu_x": float(mu[0]), "mu_y": float(mu[1]), "mu_z": float(mu[2])}


Cell 4 — Plot 3 panel

In [4]:
import matplotlib.pyplot as plt

# Warna titik beda per panel (sesuai permintaan Anda)
COLOR_BASE = "tab:blue"
COLOR_FPS  = "tab:orange"
COLOR_WRS  = "tab:green"

def apply_lims(ax, lims):
    (xmin, xmax), (ymin, ymax), (zmin, zmax) = lims
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_zlim(zmin, zmax)
    ax.set_xlabel("x"); ax.set_ylabel("y"); ax.set_zlabel("z")

def plot_three(frame_id: int):
    fb = get_frame(df_base, frame_id)
    ff = get_frame(df_fps,  frame_id)
    fw = get_frame(df_wrs,  frame_id)

    lims = compute_union_lims([fb, ff, fw])

    fig = plt.figure(figsize=(16, 5))
    ax1 = fig.add_subplot(1,3,1, projection="3d")
    ax2 = fig.add_subplot(1,3,2, projection="3d")
    ax3 = fig.add_subplot(1,3,3, projection="3d")

    if len(fb) > 0:
        ax1.scatter(fb["x"], fb["y"], fb["z"], s=6, c=COLOR_BASE)
    if len(ff) > 0:
        ax2.scatter(ff["x"], ff["y"], ff["z"], s=6, c=COLOR_FPS)
    if len(fw) > 0:
        ax3.scatter(fw["x"], fw["y"], fw["z"], s=6, c=COLOR_WRS)

    ax1.set_title(f"Baseline Trial-2 (Head-1)\n{SUBJECT}/{FILE} | frame={frame_id} | N={N_TARGET}")
    ax2.set_title(f"FPS\n{SUBJECT}/{FILE} | frame={frame_id} | N={N_TARGET}")
    ax3.set_title(f"WRS\n{SUBJECT}/{FILE} | frame={frame_id} | N={N_TARGET}")

    for ax in (ax1, ax2, ax3):
        apply_lims(ax, lims)
        ax.view_init(elev=18, azim=-55)  # view angle sama

    plt.tight_layout()
    plt.show()

    # tabel ringkas (n + centroid) untuk sanity visual
    stats = pd.DataFrame([
        {"set":"baseline", **summarize_frame(fb)},
        {"set":"fps",      **summarize_frame(ff)},
        {"set":"wrs",      **summarize_frame(fw)},
    ])
    display(stats)


Cell 5 — UI minimal: slider + input angka

In [5]:
import ipywidgets as widgets
from IPython.display import display, clear_output

frame_slider = widgets.IntSlider(
    value=fmin, min=fmin, max=fmax, step=1,
    description="Frame", continuous_update=False
)
frame_text = widgets.BoundedIntText(
    value=fmin, min=fmin, max=fmax,
    description="Go to"
)
out = widgets.Output()

_updating = False

def sync_text_from_slider(change):
    global _updating
    if _updating: return
    _updating = True
    frame_text.value = int(change["new"])
    _updating = False

def sync_slider_from_text(change):
    global _updating
    if _updating: return
    _updating = True
    frame_slider.value = int(change["new"])
    _updating = False

def render(change=None):
    with out:
        clear_output(wait=True)
        plot_three(int(frame_slider.value))

frame_slider.observe(sync_text_from_slider, names="value")
frame_text.observe(sync_slider_from_text, names="value")

frame_slider.observe(render, names="value")

display(widgets.HBox([frame_slider, frame_text]))
display(out)

# initial render
render()


Output()

Cell 6 — 2D Plot

In [6]:
# =========================
# 2D VIEW SETTINGS (EDIT HERE)
# =========================
PLANE_2D = "XY"   # opsi: "XY", "XZ", "YZ"

COLOR_BASE = "tab:blue"
COLOR_FPS  = "tab:orange"
COLOR_WRS  = "tab:green"

import matplotlib.pyplot as plt
from IPython.display import clear_output, display
import pandas as pd
import numpy as np

out2d = widgets.Output()

def _axes_from_plane(plane: str):
    plane = plane.upper()
    if plane == "XY":
        return ("x", "y")
    if plane == "XZ":
        return ("x", "z")
    if plane == "YZ":
        return ("y", "z")
    raise ValueError("PLANE_2D must be one of: 'XY', 'XZ', 'YZ'")

def _compute_union_lims_2d(dfs, ax_a, ax_b, margin_ratio=0.10):
    pts = []
    for d in dfs:
        if d is not None and len(d) > 0:
            pts.append(d[[ax_a, ax_b]].to_numpy(dtype=np.float32))
    if not pts:
        return ((-1,1),(-1,1))

    P = np.vstack(pts)
    mn = P.min(axis=0)
    mx = P.max(axis=0)
    span = (mx - mn) + 1e-6
    margin = span * margin_ratio
    mn2 = mn - margin
    mx2 = mx + margin
    return ((float(mn2[0]), float(mx2[0])), (float(mn2[1]), float(mx2[1])))

def plot_three_2d(frame_id: int, plane: str = "XY"):
    ax_a, ax_b = _axes_from_plane(plane)

    fb = get_frame(df_base, frame_id)
    ff = get_frame(df_fps,  frame_id)
    fw = get_frame(df_wrs,  frame_id)

    lims2d = _compute_union_lims_2d([fb, ff, fw], ax_a, ax_b)

    fig, axes = plt.subplots(1, 3, figsize=(16, 4))

    # Baseline
    if len(fb) > 0:
        axes[0].scatter(fb[ax_a], fb[ax_b], s=8, c=COLOR_BASE)
    axes[0].set_title(f"Baseline (Trial-2)\n{plane} | frame={frame_id} | N={N_TARGET}")
    axes[0].set_xlabel(ax_a); axes[0].set_ylabel(ax_b)

    # FPS
    if len(ff) > 0:
        axes[1].scatter(ff[ax_a], ff[ax_b], s=8, c=COLOR_FPS)
    axes[1].set_title(f"FPS\n{plane} | frame={frame_id} | N={N_TARGET}")
    axes[1].set_xlabel(ax_a); axes[1].set_ylabel(ax_b)

    # WRS
    if len(fw) > 0:
        axes[2].scatter(fw[ax_a], fw[ax_b], s=8, c=COLOR_WRS)
    axes[2].set_title(f"WRS\n{plane} | frame={frame_id} | N={N_TARGET}")
    axes[2].set_xlabel(ax_a); axes[2].set_ylabel(ax_b)

    # shared limits
    (a_min, a_max), (b_min, b_max) = lims2d
    for ax in axes:
        ax.set_xlim(a_min, a_max)
        ax.set_ylim(b_min, b_max)
        ax.grid(True, alpha=0.25)

    plt.tight_layout()
    plt.show()

    # stats kecil (n + centroid 3D) tetap berguna walau plot 2D
    stats = pd.DataFrame([
        {"set":"baseline", **summarize_frame(fb)},
        {"set":"fps",      **summarize_frame(ff)},
        {"set":"wrs",      **summarize_frame(fw)},
    ])
    display(stats)

def render_2d(change=None):
    with out2d:
        clear_output(wait=True)
        plot_three_2d(int(frame_slider.value), plane=PLANE_2D)

# Hook ke slider yang sama
frame_slider.observe(render_2d, names="value")

display(out2d)

# initial render
render_2d()


Output()